# Klasifikacija rijetkih biljaka i životinja

# Program

In [ ]:
## Importing useful libraries
import os
import math
import random
import numpy as np
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as data, DataLoader, Dataset
import torchvision.transforms as transforms
import torchvision
from torchsummary import summary
from torchvision import models
import torch.optim as optim


import matplotlib.pyplot as plt
from imageio.v2 import imread
%matplotlib inline

from tqdm.notebook import tqdm

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device", device)

from sklearn.metrics import accuracy_score, precision_score, recall_score
from torch.utils.tensorboard import SummaryWriter

%load_ext tensorboard

In [ ]:
# Define a Siamese network
class SiameseNetwork(nn.Module):
    def __init__(self, base_model):
        super(SiameseNetwork, self).__init__()
        self.base_model = base_model
        self.fc = nn.Linear(base_model.fc.in_features, 128)

    def forward_once(self, x):
        x = self.base_model(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

    def forward(self, input1, input2):
        output1 = self.forward_once(input1)
        output2 = self.forward_once(input2)
        return output1, output2

In [ ]:
# TODO: prilagoditi funkciju datasetu

class FewShotDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# TODO: proučiti i doraditi funkciju po uzoru na labose

def train_few_shot(model, dataloader, optimizer, criterion, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for batch_idx, (data, target) in enumerate(dataloader):
            optimizer.zero_grad()
            img1, img2, label = data
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)

            output1, output2 = model(img1, img2)
            loss = criterion(output1, output2, label)
            total_loss += loss.item()

            loss.backward()
            optimizer.step()

            if batch_idx % 100 == 0:
                print('Epoch {}, Batch {}, Loss: {:.4f}'.format(epoch, batch_idx, loss.item()))

        print('Epoch {}, Average Loss: {:.4f}'.format(epoch, total_loss / len(dataloader)))

In [ ]:
# TODO: podijeliti na train i eval funkcije
# TODO: optimizator, loss, dataset ...

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load pre-trained ResNet model   
resnet = models.resnet18(pretrained=True)
base_model = nn.Sequential(*list(resnet.children())[:-1])  # Remove the final fully connected layer

# Initialize Siamese network
siamese_model = SiameseNetwork(base_model)
siamese_model.to(device)

# Define your criterion, optimizer, and other hyperparameters
criterion = nn.TripletMarginLoss()
optimizer = optim.Adam(siamese_model.parameters(), lr=0.001)

# Example dataset - you should replace this with your own dataset
# Here, data is assumed to be a list of tuples (image, label)
data = [(torch.randn(3, 224, 224), random.randint(0, 9)) for _ in range(100)]

# TODO: augmentacija dataseta --> dodati u funk za dataset
# Transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Split data into train and test sets
train_data = data[:80]
test_data = data[80:]

# Create datasets and dataloaders
train_dataset = FewShotDataset(train_data, transform=transform)
test_dataset = FewShotDataset(test_data, transform=transform)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Train the model
train_few_shot(siamese_model, train_dataloader, optimizer, criterion, epochs=5)

# Evaluate the model
# (You can implement evaluation logic similarly to training logic, using test_dataloader)

In [ ]:
# TODO: tensorboard za vizualizaciju + dodatne funkcije po potrebi